In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
def f():
    print("-------------------------")

In [15]:
df = pd.read_csv("data.csv")
f()
print(f"{df.shape[0]} rows\n{df.shape[1]} columns")

-------------------------
2803 rows
1383 columns


In [16]:
f()
print(f"Columns types distribution:\n\n{df.dtypes.value_counts()}")

-------------------------
Columns types distribution:

float64    1378
object        3
int64         2
dtype: int64


In [30]:
f()
# a = df.isna().sum().where(lambda x:x>0).dropna()
print(f"There are {len(a)} (out of {df.s}) columns that contains 1 or more NA")

-------------------------
There are 33 columns that contains 1 or more NA
